In [1]:
from bs4 import BeautifulSoup
import html.parser
import json
import csv
import time
import re
import requests

In [2]:
h = html.parser

In [29]:
data = {"skuId":1442771,"skuCode":1189895,"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}

In [30]:
data["skuCode"] = int(sku)      

In [31]:
dataskuId

{'skuId': 1442771,
 'skuCode': '1189895',
 'countryCode': 'CY',
 'zipCode': '',
 'maxDistance': 1000000,
 'ignoreAvailability': 'true',
 'geoLocationWithZipCode': 'true',
 'topAvailableWarehouse': 100000,
 'hasGeolocation': 'false',
 'latitude': 0,
 'longitude': 0}

In [9]:
soup = BeautifulSoup(r_2.text, 'html.parser')

In [33]:
versions = soup.select('[class="versions-container"] [class*="x-product-details-version-selector"] li')
for version in versions:
    size  = str(version.select_one('[class="size"]').text).strip()
    skuId = str(version["data-value"]).strip()
    
    data = {"skuId":int(skuId),"skuCode":int(sku),"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}
    data = json.dumps(data)
    print(data)

{"skuId": 1442771, "skuCode": 1189895, "countryCode": "CY", "zipCode": "", "maxDistance": 1000000, "ignoreAvailability": "true", "geoLocationWithZipCode": "true", "topAvailableWarehouse": 100000, "hasGeolocation": "false", "latitude": 0, "longitude": 0}
{"skuId": 1442772, "skuCode": 1189895, "countryCode": "CY", "zipCode": "", "maxDistance": 1000000, "ignoreAvailability": "true", "geoLocationWithZipCode": "true", "topAvailableWarehouse": 100000, "hasGeolocation": "false", "latitude": 0, "longitude": 0}


In [7]:
url = "https://www.e-jumbo.gr/eidi-pet-shop/volta-skylou/samarakia-skylou/samaraki-anapaftiko-bez-kafe-small_1349348/?country=CY"

In [39]:
headers = {
    'authority': 'www.e-jumbo.gr',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'origin': 'https://www.e-jumbo.gr',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': url,
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
}

params = (
    ('lang', 'el'),
)


r_1 = requests.get(url, headers=headers)
soup = BeautifulSoup(r_1.text, 'html.parser')

name = str(soup.select_one('h1[itemprop="name"]').text).strip()

try:
    price = str(soup.select_one('[class="current-price"]').text).strip()
except:
    price = ""
    
try:    
    sku  = str(soup.select_one('[class="code"]').text).replace("Κωδικός Jumbo:","").strip()
except:
    sku  = ""
    
try:
    category = ""
    cats = soup.select('[class="breadcrumb clearfix"] [class="title"]')
    for cat in cats[:4]:
        category += str(cat.text).strip() + " > "
    category = category.strip(" > ")
except:
    category = ""
    
try:
    images = ""
    images_raw=soup.select('a[class="cloud-zoom"]')
    for img in images_raw:
        images += "https:" + str(img["href"]).strip().split(".jpg?")[0] + ".jpg" + "\n"
    images = images.strip("\n") 
    #print(images)
except:
    images = ""

try:
    description = ""
    sections = soup.select('[class="tabs-content"] [class*="tab-box"]')
    for sec in sections:
        label = str(sec.select_one('[class="title"]').text).strip()
        if label =="Περιγραφή - Χαρακτηριστικά":
            description = str(sec.select_one('[class="dynamic-content"]').text).strip()
except:
    description = ""

f_data = [url,name,description,sku,"","",images,price,category]

with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
    csv_writer = csv.writer(new_file)
    csv_writer.writerow(f_data)    
    
versions = soup.select('[class="versions-container"] [class*="x-product-details-version-selector"] li')
skus = str(soup.select_one('[class*="ProductAttributes_ExtraSearchText"] [class*="att-cell-value"]').text).strip().split(" ")

for j in range(len(versions)):
    sku     = skus[j]
    version = versions[j]
    size  = str(version.select_one('[class="size"]').text).strip()
    skuId = str(version["data-value"]).strip()
    
    data = {"skuId":int(skuId),"skuCode":int(sku),"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}
    data = json.dumps(data)
    print(data)
    
    r_2 = requests.post('https://www.e-jumbo.gr/services/EcomService.svc/GetProductAvailableWarehouses', headers=headers, params=params, data=data)
    w_data = json.loads(r_2.json()["d"]["data"])
    
    availability = "" 
                        
    try:
        warehouses   = w_data["WarehousesGrouped"][0]["Items"]
    except:
        warehouses   =  []
        availability = "Available in Online"
        
    for wh in warehouses:
        area = str(wh["Area"]).strip()
        if area == "Λάρνακα":
            availability = str(wh["AvailabilityName"]).strip()
            
    f_data = ["","","","",size,availability,"","",""]

    with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
        csv_writer = csv.writer(new_file)
        csv_writer.writerow(f_data)
    print(availability)

{"skuId": 278720, "skuCode": 1147416, "countryCode": "CY", "zipCode": "", "maxDistance": 1000000, "ignoreAvailability": "true", "geoLocationWithZipCode": "true", "topAvailableWarehouse": 100000, "hasGeolocation": "false", "latitude": 0, "longitude": 0}
Διαθέσιμο
{"skuId": 335476, "skuCode": 1151833, "countryCode": "CY", "zipCode": "", "maxDistance": 1000000, "ignoreAvailability": "true", "geoLocationWithZipCode": "true", "topAvailableWarehouse": 100000, "hasGeolocation": "false", "latitude": 0, "longitude": 0}
Available in Online


In [38]:
skus = str(soup.select_one('[class*="ProductAttributes_ExtraSearchText"] [class*="att-cell-value"]').text).strip().split(" ")
skus

['1147416', '1151833']

In [81]:
images_raw=soup.select('a[class="cloud-zoom"]')
images = ""
for img in images_raw:
    images += "https:" + str(img["href"]).strip().split(".jpg?")[0] + ".jpg" + "\n"
images = images.strip("\n") 
print(images)

https://static-v3.e-jumbo.gr/uploads/resources/222677/500501000030_01-huge.jpg
https://static-v3.e-jumbo.gr/uploads/resources/222678/500501000030_02-huge.jpg
https://static-v3.e-jumbo.gr/uploads/resources/222679/500501000030_03-huge.jpg
https://static-v3.e-jumbo.gr/uploads/resources/222680/500501000030_04-huge.jpg


In [43]:
r_2.json()

{'d': {'__type': 'ServiceResponseOfCommonResponseTypeWH_P1lev_S:#Lighthouse.Lh3.FrontEnd.Services.Endpoints',
  'code': 'OperationSuccesful',
  'data': '{"StoresText":"ΚΑΤΑΣΤΗΜΑ","AvailabilityText":"ΔΙΑΘΕΣΙΜΟΤΗΤΑ","NoWarehouseFound":"Δεν βρέθηκαν καταστήματα","WarehousesGrouped":[]}',
  'email': None,
  'success': True,
  'time': 0}}

In [42]:
w_data = json.loads(r_2.json()["d"]["data"])
w_data 

{'StoresText': 'ΚΑΤΑΣΤΗΜΑ',
 'AvailabilityText': 'ΔΙΑΘΕΣΙΜΟΤΗΤΑ',
 'NoWarehouseFound': 'Δεν βρέθηκαν καταστήματα',
 'WarehousesGrouped': []}

In [41]:
warehouses = w_data["WarehousesGrouped"][0]["Items"]

In [47]:
for wh in warehouses:
    area = str(wh["Area"]).strip()
    if area == "Λάρνακα":
        availability = str(wh["AvailabilityName"]).strip()
print(availability)

Διαθέσιμο


In [ ]:
import requests

cookies = {
    'ForceClearDeliveryPointLocalStorage': '1',
    'countrySelectorPopupOpened': 'true',
    '_gcl_au': '1.1.660232412.1623524496',
    '_fbp': 'fb.1.1623524496867.485701904',
    '_ga': 'GA1.2.2073830231.1623525700',
    '_gid': 'GA1.2.1804191354.1623525700',
    'sessionid': '2d51900131184489bc827c72e3a15ced',
    'uid': '844ed38482c24916be65f6962a6f7c77',
    'SpPlList_1354735': 'LastProductId=269237',
    'ASP.NET_SessionId': 'ttsvekzg0evd0tsuydaixfi1',
    'ROUTEID.cd107d9706d71153bafd4ab15f1c6b5d': '.node3',
    'newsletterpopup': 'true',
    'PreviousCountryCode': 'CY',
    'VisitorState': 'siteId=1&countryCode=cy&languageCode=el&currencyCode=eur&partnerId=&futureDeliveryDateTime=&deliveryPointId=&userRoleIds=',
    'CdnVisitorState': 'countryCode=cy&currencyCode=eur',
    'VisitorGUID': '4aac7d5a-4a57-4044-a795-887ab921ad6d',
    'ConsentChecked': '{"userHasSetCookies":true,"functionalityCookies":true,"statisticCookies":true,"marketingCookies":true}',
    '__atuvc': '18%7C24',
    '__atuvs': '60c64fba915f7c56005',
    '__atssc': 'skype%3B6',
    'LastViewedProducts': '1409782,1442892,1431520,1445696',
    '_gat': '1',
    'SpPlList_1354737': 'LastProductId=1517918',
    'exitIntentFlag': 'true',
    'RT': 'sl=6&ss=1623609270243&tt=61599&obo=0&sh=1623611185639%3D6%3A0%3A61599%2C1623611165256%3D5%3A0%3A52319%2C1623611154940%3D4%3A0%3A48899%2C1623611109832%3D3%3A0%3A46013%2C1623610558652%3D2%3A0%3A39210&dm=www.e-jumbo.gr&si=7f1cf869-cefb-4479-b988-10df04ae4ba0&bcn=%2F%2F684fc53e.akstat.io%2F&rl=1&ld=1623611185640&nu=https%3A%2F%2Fwww.e-jumbo.gr%2F&cl=1623611199437',
}

headers = {
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'Referer': 'https://www.e-jumbo.gr/themes/templatetheme/dist/styles-generic.css?8m7a1t5r',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
    'authority': 'www.google.co.in',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
    'accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-dest': 'image',
    'referer': 'https://www.e-jumbo.gr/',
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
    'cookie': '__Secure-3PSID=-Qe_HfXmBTvemM-K7uXYlnd9d_6ylOUlJ6p6yyXGLbzRln9QWsZkVWoHQtYUk2l3vW6LAA.; __Secure-3PAPISID=rHLM80cHqTPjgriy/AWZCkAkgAKpQXZH3s; NID=216=PZsPtOmhlpvV0_7SFqjG1E8E9ctzsfPEvUUXgphqqLUU6Puyw02ult6S3gXuYjdYkcl4Trytaa2oHIc0uS4FZ74M4Na2Upoi00d26jkr9WFiFkYxXsDvqrYSmY8TbPg1pQTECSSBYoRz6HMdTSOYqmjCY4XBYNl9AUtu2wthEcInAZOcBlVJJI97T-9cnl4601zRyhhKpIAe28mVXY0wMXZzwcWmPJjS8so_63TLjZtJO22tZ9K6YvDxh6k9yc2XLijoKwmF-W0_NIgbaw0kL9xs1g',
    'x-requested-with': 'XMLHttpRequest',
    'content-type': 'application/json; charset=UTF-8',
    'origin': 'https://www.e-jumbo.gr',
    'x-client-data': 'CKO1yQEIkLbJAQiltskBCMS2yQEIqZ3KAQigoMsBCMCgywEI3fLLAQie+MsBCLP4ywE=',
}

data = '{"skuId":1409921,"skuCode":1183758,"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":true,"geoLocationWithZipCode":true,"topAvailableWarehouse":100000,"hasGeolocation":false,"latitude":0,"longitude":0}'

r= requests.post('https://www.googleadservices.com/pagead/conversion_async.js', headers=headers, cookies=cookies, data=data)


In [ ]:
versions = '[class="versions-container"] [class*="x-product-details-version-selector"] li'

In [64]:
import pandas as pd
import numpy as np

In [71]:
# Reading the csv file
df_new = pd.read_csv('output_90.csv')
  
# saving xlsx file
GFG = pd.ExcelWriter('output_90.xlsx')
df_new.to_excel(GFG, index = False)
  
GFG.save()

In [72]:
GFG.close()

In [33]:
import requests


headers = {
    'authority': 'www.e-jumbo.gr',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'origin': 'https://www.e-jumbo.gr',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.e-jumbo.gr/eidi-pet-shop/volta-skylou/samarakia-skylou/samaraki-anapaftiko-bez-kafe-small_1349348/?country=CY',
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
}

params = (
    ('lang', 'el'),
    ('country', 'cy'),
)

data = {
  '{"serviceConfiguration":"{\\"__type\\":\\"Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version': '2020.4.0.72, Culture=neutral, PublicKeyToken=null\\",\\"LoaderOptions\\":{\\"__type\\":\\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\\",\\"ParentProductId\\":1349348,\\"PathChecksum\\":-17341838,\\"LanguageId\\":1,\\"SkuId\\":335476,\\"SplitDimensions\\":null,\\"BundleItems\\":[],\\"ItemAttributes\\":[],\\"LanguageCode\\":\\"el\\",\\"CurrencyCode\\":\\"EUR\\",\\"SiteCode\\":\\"www-e-jumbo-gr\\",\\"Cacher\\":null},\\"AdapterOptions\\":{\\"__type\\":\\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\\",\\"IsDynamic\\":false,\\"EnableNoImage\\":true,\\"DimensionImageTypes\\":[],\\"SelectDefaultSku\\":false,\\"Actions\\":2147483647,\\"LazyLoadPrices\\":false,\\"AttributesStartIndex\\":null,\\"Articles\\":{},\\"Literals\\":{\\"QuantityText\\":\\"\u03A0\u039F\u03A3\u039F\u03A4\u0397\u03A4\u0391:\\",\\"VatText\\":\\"\u03A3\u03C4\u03B7\u03BD \u03C4\u03B9\u03BC\u03AE \u03C3\u03C5\u03BC\u03C0\u03B5\u03C1\u03B9\u03BB\u03B1\u03BC\u03B2\u03AC\u03BD\u03B5\u03C4\u03B1\u03B9 \u03BF \u03A6.\u03A0.A.\\",\\"ShopOnlyProductBtnText\\":\\"\u0395\u03A0\u0399\u039A\u039F\u0399\u039D\u03A9\u039D\u0397\u03A3\u03A4\u0395 \u039C\u0395 \u039A\u0391\u03A4\u0391\u03A3\u03A4\u0397\u039C\u0391\\",\\"ShopOnlyProductBtnText2\\":\\"\u03B2\u03C1\u03B5\u03C2 \u03C4\u03BF \u03C0\u03BB\u03B7\u03C3\u03B9\u03AD\u03C3\u03C4\u03B5\u03C1\u03BF\\",\\"StoresLocatorUrl\\":\\"http://corporate.e-jumbo.gr/katastimata-jumbo/\\",\\"ShopOnlyProductBtnTextUrl\\":\\"\u03B5\u03B4\u03CE\\",\\"IsBatteryRequiredText\\":\\"\u039B\u0395\u0399\u03A4\u039F\u03A5\u03A1\u0393\u0395\u0399 \u039C\u0395 \u039C\u03A0\u0391\u03A4\u0391\u03A1\u0399\u0395\u03A3\\",\\"BatteryBuyText\\":\\"\u0394\u0395\u0399\u03A4\u0395 \u0395\u0394\u03A9\\",\\"IsBatteryRequiredUrl\\":\\"/paichnidia/bataries-alkalikes/\\",\\"SafeProductTitle\\":\\"\u039A\u0391\u03A4\u0391\u039B\u039B\u0397\u039B\u039F\u03A4\u0397\u03A4\u0391 \u03A0\u03A1\u039F\u03AA\u039F\u039D\u03A4\u039F\u03A3\\",\\"PriceText\\":\\"\u03A4\u0399\u039C\u0397 JUMBO:\\",\\"AvailabilityText\\":\\"\u0394\u03B9\u03B1\u03B8\u03B5\u03C3\u03B9\u03BC\u03CC\u03C4\u03B7\u03C4\u03B1:\\",\\"OrderedWithIconClass\\":\\"rocket\\",\\"GalleryText\\":\\"\u03A4\u03BF\u03C0\u03BF\u03B8\u03B5\u03C4\u03AE\u03C3\u03C4\u03B5 \u03C4\u03BF\u03BD \u03BA\u03AD\u03C1\u03C3\u03BF\u03C1\u03B1 \u03C3\u03C4\u03B7 \u03C6\u03C9\u03C4\u03BF\u03B3\u03C1\u03B1\u03C6\u03AF\u03B1 \u03B3\u03B9\u03B1 \u03BC\u03B5\u03B3\u03AD\u03B8\u03C5\u03BD\u03C3\u03B7\\",\\"AssortmentCodesText\\":\\"\u03A4\u03BF \u03C0\u03C1\u03BF\u03CA\u03CC\u03BD \u03B5\u03AF\u03BD\u03B1\u03B9 \u03BA\u03C9\u03B4\u03B9\u03BA\u03CC\u03C2 \u03C4\u03C5\u03C7\u03B1\u03AF\u03B1\u03C2 \u03B5\u03C0\u03B9\u03BB\u03BF\u03B3\u03AE\u03C2, \u03B4\u03B7\u03BB\u03B1\u03B4\u03AE \u03B4\u03B9\u03B1\u03C4\u03AF\u03B8\u03B5\u03C4\u03B1\u03B9 \u03C3\u03B5 \u03C0\u03B1\u03C1\u03B1\u03C0\u03AC\u03BD\u03C9 \u03B1\u03C0\u03CC \u03AD\u03BD\u03B1 \u03C3\u03C7\u03AD\u03B4\u03B9\u03B1 \u03AE \u03C7\u03C1\u03CE\u03BC\u03B1\u03C4\u03B1. \u0394\u03B5\u03AF\u03C4\u03B5 \u03C4\u03BF\u03C5\u03C2 \u03CC\u03C1\u03BF\u03C5\u03C2 \u03BA\u03B1\u03B9 \u03C0\u03C1\u03BF\u03CB\u03C0\u03BF\u03B8\u03AD\u03C3\u03B5\u03B9\u03C2 \u03B5\u03C0\u03B9\u03C3\u03C4\u03C1\u03BF\u03C6\u03CE\u03BD \u03BA\u03C9\u03B4\u03B9\u03BA\u03CE\u03BD \u03C4\u03C5\u03C7\u03B1\u03AF\u03B1\u03C2 \u03B5\u03C0\u03B9\u03BB\u03BF\u03B3\u03AE\u03C2.\\",\\"DescriptionTabTitle\\":\\"Video\\",\\"VideoThumb\\":\\"VIDEO\\",\\"SafeClothingTitle\\":\\"\u03A0\u0399\u03A3\u03A4\u039F\u03A0\u039F\u0399\u0397\u03A3\u0397\\",\\"SafeClothingTooltip\\":\\"\u0388\u03C7\u03B5\u03B9 \u03B5\u03BB\u03B5\u03B3\u03C7\u03B8\u03B5\u03AF \u03B3\u03B9\u03B1 \u03B2\u03BB\u03B1\u03B2\u03B5\u03C1\u03AD\u03C2 \u03BF\u03C5\u03C3\u03AF\u03B5\u03C2\\",\\"SafeDishwasherTooltip\\":\\"\u03A4\u03BF \u03C0\u03C1\u03BF\u03CA\u03CC\u03BD <span>\u03B4\u03B5\u03BD</span> \u03B5\u03AF\u03BD\u03B1\u03B9 \u03BA\u03B1\u03C4\u03AC\u03BB\u03BB\u03B7\u03BB\u03BF \u03B3\u03B9\u03B1 \u03C0\u03BB\u03C5\u03BD\u03C4\u03AE\u03C1\u03B9\u03BF \u03C0\u03B9\u03AC\u03C4\u03C9\u03BD\\",\\"SafeMicrowaveTooltip\\":\\"\u03A4\u03BF \u03C0\u03C1\u03BF\u03CA\u03CC\u03BD <span>\u03B4\u03B5\u03BD</span> \u03B5\u03AF\u03BD\u03B1\u03B9 \u03BA\u03B1\u03C4\u03AC\u03BB\u03BB\u03B7\u03BB\u03BF \u03B3\u03B9\u03B1 \u03C6\u03BF\u03CD\u03C1\u03BD\u03BF \u03BC\u03B9\u03BA\u03C1\u03BF\u03BA\u03C5\u03BC\u03AC\u03C4\u03C9\u03BD\\",\\"SuitableTitle\\":\\"\u039A\u0391\u03A4\u0391\u039B\u039B\u0397\u039B\u039F\u03A4\u0397\u03A4\u0391\\",\\"WarehouseTooltip\\":\\"\u0395\u03C0\u03B9\u03BB\u03AD\u03BE\u03C4\u03B5 \u03C0\u03C1\u03CE\u03C4\u03B1 \u03BC\u03AD\u03B3\u03B5\u03B8\u03BF\u03C2 \u03B3\u03B9\u03B1 \u03BD\u03B1 \u03B4\u03B5\u03AF\u03C4\u03B5 \u03C4\u03B7\u03BD \u03B4\u03B9\u03B1\u03B8\u03B5\u03C3\u03B9\u03BC\u03CC\u03C4\u03B7\u03C4\u03B1 \u03C3\u03C4\u03B1 \u03BA\u03B1\u03C4\u03B1\u03C3\u03C4\u03AE\u03BC\u03B1\u03C4\u03B1\\",\\"AvailabilityLabel\\":\\"\u0395\u03C0\u03B9\u03BB\u03AD\u03BE\u03C4\u03B5 \u03BC\u03AD\u03B3\u03B5\u03B8\u03BF\u03C2 \u03B3\u03B9\u03B1 \u03BD\u03B1 \u03B4\u03B5\u03AF\u03C4\u03B5 \u03C4\u03B7\u03BD \u03B4\u03B9\u03B1\u03B8\u03B5\u03C3\u03B9\u03BC\u03CC\u03C4\u03B7\u03C4\u03B1\\",\\"TabOneLabel\\":\\"\u03A0\u03B5\u03C1\u03B9\u03B3\u03C1\u03B1\u03C6\u03AE\\",\\"TabTwoLabel\\":\\"\u039C\u03B5\u03C4\u03B1\u03C6\u03BF\u03C1\u03B9\u03BA\u03AC \u0388\u03BE\u03BF\u03B4\u03B1\\",\\"TabThreeLabel\\":\\"\u0395\u03C0\u03B9\u03C3\u03C4\u03C1\u03BF\u03C6\u03AD\u03C2\\",\\"TabFourLabel\\":\\"\u03A7\u03C1\u03CC\u03BD\u03BF\u03B9 \u03A0\u03B1\u03C1\u03AC\u03B4\u03BF\u03C3\u03B7\u03C2\\",\\"SizeGuideLabel\\":\\"\u0392\u03C1\u03B5\u03C2 \u03C4\u03BF \u03BC\u03AD\u03B3\u03B5\u03B8\u03BF\u03C2 \u03C3\u03BF\u03C5 \u03B5\u03B4\u03CE\\",\\"CanonicalLabel\\":\\"\u0394\u03B5\u03AF\u03C4\u03B5 \u03C4\u03B1 \u03CC\u03BB\u03B1 \u03B5\u03B4\u03CE\\",\\"InstructionsPdfTitle\\":\\"\u039F\u03B4\u03B7\u03B3\u03AF\u03B5\u03C2 \u03C7\u03C1\u03AE\u03C3\u03B5\u03C9\u03C2-\u03C3\u03C5\u03BD\u03B1\u03C1\u03BC\u03BF\u03BB\u03CC\u03B3\u03B7\u03C3\u03B7\u03C2 \\",\\"InstructionsPdfUrl\\":\\"\u039B\u03AE\u03C8\u03B7 \u03B1\u03C1\u03C7\u03B5\u03AF\u03BF\u03C5 PDF\\"},\\"SocialShares\\":null,\\"ProductReview\\":null,\\"WarehouseAvailabilities\\":{\\"__type\\":\\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductWarehouseAvailabilityDataViewModel, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\\",\\"GeoLocationWithZipCode\\":true,\\"IgnoreAvailability\\":true,\\"WarehouseResultsCount\\":100000,\\"PartnerGroupBy\\":\\"\\",\\"SelectStores\\":false,\\"EnableTrackingLocation\\":false,\\"WarehouseDistances\\":[{\\"__type\\":\\"Lighthouse.Lh3.FrontEnd.Root.Model.DistanceSetValueOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\\",\\"Value\\":\\"1000000\\",\\"Name\\":\\"\u03C7\u03C9\u03C1\u03AF\u03C2 \u03CC\u03C1\u03B9\u03BF\\"}],\\"CheckAvailabilityText\\":\\"\u0391\u039D\u0391\u0396\u0397\u03A4\u0397\u03A3\u0397 \u039A\u0391\u03A4\u0391\u03A3\u03A4\u0397\u039C\u0391\u03A4\u03A9\u039D\\",\\"CheckAvailabilityWithTrackingLocationText\\":\\"Search Warehouses Near My Location\\",\\"ShowWarehouseSearchText\\":\\"\u0395\u039B\u0395\u0393\u03A7\u039F\u03A3 \u0394\u0399\u0391\u0398\u0395\u03A3\u0399\u039C\u039F\u03A4\u0397\u03A4\u0391\u03A3 \u03A3\u03A4\u0391 \u039A\u0391\u03A4\u0391\u03A3\u03A4\u0397\u039C\u0391\u03A4\u0391\\",\\"WarehouseHelpText\\":\\"\u0395\u03B9\u03C3\u03AC\u03B3\u03B5\u03C4\u03B5 \u03C4\u03BF\u03BD \u03A4\u039A \u03C3\u03B1\u03C2 \u03BA\u03B1\u03B9 \u03B4\u03B5\u03AF\u03C4\u03B5 \u03C4\u03B7 \u03B4\u03B9\u03B1\u03B8\u03B5\u03C3\u03B9\u03BC\u03CC\u03C4\u03B7\u03C4\u03B1 \u03C3\u03C4\u03B1 \u03BA\u03B1\u03C4\u03B1\u03C3\u03C4\u03AE\u03BC\u03B1\u03C4\u03B1 Jumbo \u03C0\u03BF\u03C5 \u03B2\u03C1\u03AF\u03C3\u03BA\u03BF\u03BD\u03C4\u03B1\u03B9 \u03B4\u03AF\u03C0\u03BB\u03B1 \u03C3\u03B1\u03C2.\\",\\"WarehouseHelpTextTwo\\":\\"\u0395\u03BB\u03AD\u03B3\u03BE\u03C4\u03B5 \u03C4\u03B7\u03BD \u03B4\u03B9\u03B1\u03B8\u03B5\u03C3\u03B9\u03BC\u03CC\u03C4\u03B7\u03C4\u03B1 \u03C4\u03BF\u03C5 \u03C0\u03C1\u03BF\u03CA\u03CC\u03BD\u03C4\u03BF\u03C2 \u03C3\u03C4\u03BF \u03BA\u03B1\u03C4\u03AC\u03C3\u03C4\u03B7\u03BC\u03B1 \u03C4\u03B7\u03C2 \u03B5\u03C0\u03B9\u03BB\u03BF\u03B3\u03AE\u03C2 \u03C3\u03B1\u03C2:\\",\\"WarehouseTitle\\":\\"\u0394\u03B9\u03B1\u03B8\u03B5\u03C3\u03B9\u03BC\u03CC\u03C4\u03B7\u03C4\u03B1 \u03C3\u03B5 \u03BA\u03B1\u03C4\u03AC\u03C3\u03C4\u03B7\u03BC\u03B1\\",\\"WarehouseSearchText\\":\\"\u0395\u03B9\u03C3\u03AC\u03B3\u03B5\u03C4\u03B5 \u03A4\u039A\\",\\"WarehouseDistancesText\\":\\"\u039C\u03AD\u03B3\u03B9\u03C3\u03C4\u03B7 \u03B1\u03C0\u03CC\u03C3\u03C4\u03B1\u03C3\u03B7\\",\\"WarehouseContinueShopping\\":\\"SELECT STORE ',
  ' CONTINUE SHOPPING\\",\\"WarehouseProceedToCheckout\\":\\"SELECT STORE ': '',
  ' PROCEED TO CHECKOUT\\",\\"LocationText\\":\\"\u0395\u03B9\u03C3\u03AC\u03B3\u03B5\u03C4\u03B5 \u03A4\u039A\\",\\"ErrorCalculatingDistancesText\\":\\"No calculation returned\\",\\"Configuration\\":null,\\"DebugInformation\\":null}},\\"LoaderAssemblyName\\":\\"Vendd.Website.Backend, Version': '2020.4.0.72, Culture=neutral, PublicKeyToken=null\\",\\"LoaderClassName\\":\\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoader\\",\\"AdapterAssemblyName\\":\\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\\",\\"AdapterClassName\\":\\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsAdapter\\"}"}'
}

response = requests.post('https://www.e-jumbo.gr/services/AdapterService.svc/GetData', headers=headers, params=params, data=data)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.post('https://www.e-jumbo.gr/services/AdapterService.svc/GetData?lang=el&country=cy', headers=headers, cookies=cookies, data=data)


In [32]:
response.text

''